# AoC 2024 - Day 13

<https://adventofcode.com/2024/day/13>

In [1]:
from icecream import ic
import time

In [2]:
use_test = True  # Comment out for using actual puzzle input

## Part 1

In [3]:
try:
    use_test
except NameError:
    use_test = False
    ic("use_test was undefined - forcing to", use_test)
    
# if use_test:
#     # assert test_data != None
#     # assert test_data2 != None
#     pass

FILENAME = "input_day13_gmacario.txt"
# FILENAME = "test_day13_gmacario.txt"

In [4]:
machine_0_config = {
    "a": (94, 34),
    "b": (22, 67),
    "p": (8400, 5400)
}

ic(machine_0_config)

ic| machine_0_config: {'a': (94, 34), 'b': (22, 67), 'p': (8400, 5400)}


{'a': (94, 34), 'b': (22, 67), 'p': (8400, 5400)}

In [5]:
# Read the puzzle input into a data structure, one per line
#

def s2i(s) -> int:
    return int(s.split(",")[0])

machines = list()
num_machines = 0
with open(FILENAME, 'r') as f:
    again = True
    while again:
        _, a_x, a_y = f.readline().split("+")
        _, b_x, b_y = f.readline().split("+")
        _, p_x, p_y = f.readline().split("=")
        entry = {
            "A": (s2i(a_x), s2i(a_y)),
            "B": (s2i(b_x), s2i(b_y)),
            "P": (s2i(p_x), s2i(p_y))
        }
        # ic(entry)
        machines.append(entry)
        num_machines += 1
        if f.readline() == "":
            again = False

ic(num_machines, machines)
ic(time.ctime())

ic| num_machines: 320
    machines: [{'A': (42, 24), 'B': (23, 55), 'P': (18849, 8875)},
               {'A': (89, 34), 'B': (35, 95), 'P': (6352, 5202)},
               {'A': (85, 18), 'B': (28, 48), 'P': (8253, 4314)},
               {'A': (74, 96), 'B': (89, 29), 'P': (12904, 11034)},
               {'A': (53, 17), 'B': (14, 43), 'P': (14746, 17141)},
               {'A': (91, 52), 'B': (35, 86), 'P': (6923, 9632)},
               {'A': (60, 46), 'B': (13, 34), 'P': (4801, 4570)},
               {'A': (48, 88), 'B': (80, 32), 'P': (6544, 3856)},
               {'A': (42, 21), 'B': (33, 52), 'P': (16097, 10266)},
               {'A': (41, 74), 'B': (94, 43), 'P': (10061, 5873)},
               {'A': (57, 41), 'B': (11, 26), 'P': (6496, 3254)},
               {'A': (14, 61), 'B': (55, 15), 'P': (4759, 4846)},
               {'A': (98, 16), 'B': (91, 98), 'P': (15407, 10414)},
               {'A': (54, 12), 'B': (22, 41), 'P': (2182, 3446)},
               {'A': (69, 25), 'B': (25, 80)

'Fri Dec 13 07:19:58 2024'

In [6]:
# def move_one_step(machine: dict, initial_pos: tuple, seq:list) -> x:
#     """
#     starting from initial_pos, move one step
#     """

In [7]:
# def find_cheapest_path(machine: dict) -> tuple:
#     (x, y) = (0, 0)

In [8]:
# Try some movement on machines[0]

mach = machines[0]
ic(mach)
pos = (0, 0)
cost = 0

# push("A")
cost += 3
new_x = pos[0] + mach['A'][0]
new_y = pos[1] + mach['A'][1]
pos = (new_x, new_y)
_ = ic("After push(A):", cost, pos)

# push("B")
cost += 1
new_x = pos[0] + mach['B'][0]
new_y = pos[1] + mach['B'][1]
pos = (new_x, new_y)
_ = ic("After push(B):", cost, pos)

ic| mach: {'A': (42, 24), 'B': (23, 55), 'P': (18849, 8875)}
ic| 'After push(A):', cost: 3, pos:

 (42, 24)
ic| 'After push(B):', cost: 4, pos: (65, 79)


In [9]:
def check_winning_sequence(mach: dict, initial_pos: tuple, num_a: int, num_b: int) -> int:
    """Find if some movement are winning
    Return the cost of movement, or -1 if it does not win
    """
    # ic(mach, initial_pos, num_a, num_b)

    if num_a > 100 or num_b > 100:
        # ic(num_a, num_b, "too big")
        return -1

    end_x = initial_pos[0]
    end_y = initial_pos[1]
    cost = 0

    end_x = end_x + mach['A'][0] * num_a
    end_y = end_y + mach['A'][1] * num_a
    cost += num_a * 3

    end_x = end_x + mach['B'][0] * num_b
    end_y = end_y + mach['B'][1] * num_b
    cost += num_b * 1

    if (end_x, end_y) == (mach['P'][0], mach['P'][1]):
        return cost
    else:
        # ic((end_x, end_y), "not in", mach['P'])
        return -1

In [11]:
# assert check_winning_sequence(machines[0], (0,0), 80, 40) == 280
# assert check_winning_sequence(machines[2], (0,0), 38, 86) == 200

In [12]:
def find_cheapest_winning_sequence(mach: dict) -> tuple:
    """
    Return a tuple (best_num_a:int, best_num_b:int, best_cost:int)
    """
    best_num_a = None
    best_num_b = None
    best_cost = None
    for num_a in range(1, 101):
        for num_b in range(1, 101):
            # ic("Trying", num_a, num_b)
            c = check_winning_sequence(mach, (0,0), num_a, num_b)
            if c < 0:
                pass
            elif best_cost is None and c > 0:
                best_num_a = num_a
                best_num_b = num_b
                best_cost = c
            elif best_cost is not None and c < best_cost:
                best_num_a = num_a
                best_num_b = num_b
                best_cost = c
                ic("Found", best_cost, "with", best_num_a, best_num_b)

    return (best_num_a, best_num_b, best_cost)

# _ = ic(find_cheapest_winning_sequence(machines[0])) # (80, 40, 280)
# assert find_cheapest_winning_sequence(machines[0]) == (80, 40, 280)
# _ = ic(find_cheapest_winning_sequence(machines[1])) # No combination
# _ = ic(find_cheapest_winning_sequence(machines[2])) # (38, 86, 200)
# _ = ic(find_cheapest_winning_sequence(machines[3])) # No combination

In [13]:
# total_tokens = 0
# for mach in machines:
#     n_a, n_b, n_tokens = find_cheapest_winning_sequence(mach)
#     ic(n_a, n_b, n_tokens)
#     if n_tokens is not None:
#         total_tokens += n_tokens

# _ = ic(total_tokens)

In [16]:
def solve_part1():
    result = 0
    for mach in machines:
        n_a, n_b, n_tokens = find_cheapest_winning_sequence(mach)
        # ic(n_a, n_b, n_tokens)
        if n_tokens is not None:
            result += n_tokens
    return result

tm_begin = time.time()
part1_result = solve_part1()
tm_end = time.time()
ic("elapsed=", tm_end - tm_begin, "result=", part1_result)
print("Day 13 Part 1 RESULT:")
print(part1_result)

ic| "elapsed=": 'elapsed='
    tm_end - tm_begin: 1.4918406009674072
    "result=": 'result='
    part1_result: 37680


Day 13 Part 1 RESULT:
37680


## Part 2

In [ ]:
disk_map = convert_input_to_map(input_lines[0],verbose=False)
print(disk_map)

NameError: name 'convert_input_to_map' is not defined

In [ ]:
# Find the largest file_id number
highest_file_id = max(((id if id != "." else -1) for id in disk_map))
ic(highest_file_id)
# num_items_with_file_id = sum((id == highest_file_id for id in disk_map))
num_items_with_file_id = disk_map.count(highest_file_id)
ic(num_items_with_file_id)

first_empty_pos = disk_map.index(".")
ic(first_empty_pos)

# disk_map = disk_map[0:first_empty_pos].extend([highest_file_id] * num_items_with_file_id)
# print(disk_map)

# next_full_pos = disk_map.search()

In [ ]:
assert False

In [ ]:
def solve_part2(input_map):
    result = set()
    for antenna_freq in get_antenna_frequencies(input_map):
        x = find_antinodes(input_map, antenna_freq, multiple=True)
        # ic(x)
        result = result.union(x)
        # ic(x, result)
    return len(result)

tm_begin = time.time()
part2_result = solve_part2(input_map)
tm_end = time.time()
ic("elapsed=", tm_end - tm_begin, "result=", part2_result)
print("Day 09 Part 2 RESULT:")
print(part2_result)